# TRY 2

In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup

import time, re

In [2]:
urls = [#common
        'https://www.bestbuy.com/site/apple-iphone-14-128gb-unlocked-midnight/6507555.p?skuId=6507555',
        'https://www.bestbuy.com/site/samsung-galaxy-s22-ultra-128gb-phantom-black-verizon/6494471.p?skuId=6494471',
        'https://www.bestbuy.com/site/hp-deskjet-2755e-wireless-inkjet-printer-with-3-months-of-instant-ink-included-with-hp-white/6454282.p?skuId=6454282',
        'https://www.bestbuy.com/site/jbl-flip-5-portable-bluetooth-speaker-black/6356535.p?skuId=6356535',
        'https://www.bestbuy.com/site/microsoft-365-family-up-to-6-people-12-month-subscription-activation-required-windows-mac-os-apple-ios-android-digital/6258038.p?skuId=6258038',
        'https://www.bestbuy.com/site/turbotax-deluxe-2023-federal-e-file-state-mac-os-windows-digital/6560825.p?skuId=6560825',
        'https://www.bestbuy.com/site/hamilton-beach-classic-4-slice-toaster-with-sure-toast-technology-stainless-steel/6518125.p?skuId=6518125',
        'https://www.bestbuy.com/site/razor-mx350-dirt-rocket-ebike-w-7-miles-max-operating-range-14-mph-max-speed-small-blue/3698469.p?skuId=3698469',
        'https://www.bestbuy.com/site/lg-65-class-c3-series-oled-4k-uhd-smart-webos-tv/6535929.p?skuId=6535929',
        'https://www.bestbuy.com/site/ge-1-4-cu-ft-mid-size-microwave-black/3180025.p?skuId=3180025',
        #BB exclusive
        'https://www.bestbuy.com/site/mcafee-total-protection-5-device-antivirus-internet-security-software-1-year-subscription-android-apple-ios-chrome-mac-os-windows-digital/6517326.p?skuId=6517326',
        'https://www.bestbuy.com/site/norton-360-deluxe-5-device-antivirus-internet-security-software-vpn-dark-web-monitoring-1-year-subscription-android-mac-os-windows-apple-ios-digital/6346690.p?skuId=6346690',
        'https://www.bestbuy.com/site/apple-iphone-13-5g-128gb-unlocked-midnight/6417788.p?skuId=6417788'
        ]

s = Service(executable_path='D:\CU Boulder - Data Science\Spring 2024\Data Mining\chromedriver.exe')

In [3]:
Global_Reviewer_name_lst = []
Global_Review_title_lst = []
Global_Reviewer_star_rating_lst = []
Global_Review_content_lst = []
Global_Helpfulness_votes_lst = []

Global_product_name_lst = []
Global_product_star_rating_lst = []
Global_product_total_reviews = []

In [5]:
for url in urls:
    driver = webdriver.Chrome(service=s)
    driver.get(url)
    driver.maximize_window()
    reviews_button = driver.find_element(By.LINK_TEXT,'See All Customer Reviews')
    reviews_button.click()

    html_product = driver.find_element(By.CLASS_NAME,'row').get_attribute('outerHTML')
    product_soup = BeautifulSoup(html_product,'html.parser')

    # Product name
    product_name = product_soup.find('a',class_='v-fw-regular').text
    # Product star rating
    product_star_rating = product_soup.find('span', class_ ='ugc-c-review-average font-weight-medium order-1').text
    # Total product reviews
    text_product_total_reviews = product_soup.find('span', class_ ='c-reviews order-2').text

    match = re.search(r'\(([\d,]+)\sReviews\)', text_product_total_reviews)
    str_product_total_reviews = match.group(1)
    product_total_reviews = int(str_product_total_reviews.replace(',', ''))

    Global_product_name_lst.append(product_name)
    Global_product_star_rating_lst.append(product_star_rating)
    Global_product_total_reviews.append(product_total_reviews)

    Reviewer_name_lst = []
    Review_title_lst = []
    Reviewer_star_rating_lst = []
    Review_content_lst = []
    Helpfulness_votes_lst = []

    n = round(product_total_reviews/20)

    for j in range(1,n+1):

        whole_page = driver.find_element(By.CLASS_NAME,'user-generated-content-ratings-and-reviews')
        time.sleep(3)
        html_review = whole_page.find_element(By.CLASS_NAME,'reviews-list').get_attribute('outerHTML')
        whole_page_soup = BeautifulSoup(html_review,'html.parser')
        all_reviews = whole_page_soup.find_all('li',class_='review-item')
        all_reviews

        for i in all_reviews:

            # Reviewer's name
            reviewer_name = i.find('div', class_='ugc-author v-fw-medium body-copy-lg').text
            # Review title
            review_title = i.find('h4',class_='c-section-title review-title heading-5 v-fw-medium').text         
            # Star rating of every review
            review_star_rating = i.find('p', class_ ='visually-hidden').text[6:7]
            # Content of the review
            review_content = i.find('p', class_='pre-white-space').text
            # Helpfulness votes
            review_helpful_votes = i.find('button',class_='c-button c-button-outline c-button-sm helpfulness-button no-margin-l').text.split('(')[1].split(')')[0]

            Reviewer_name_lst.append(reviewer_name)
            Review_title_lst.append(review_title)
            Reviewer_star_rating_lst.append(review_star_rating)
            Review_content_lst.append(review_content)
            Helpfulness_votes_lst.append(review_helpful_votes)

            Global_Reviewer_name_lst.extend(Reviewer_name_lst)
            Global_Review_title_lst.extend(Review_title_lst)
            Global_Reviewer_star_rating_lst.extend(Reviewer_star_rating_lst)
            Global_Review_content_lst.extend(Review_content_lst)
            Global_Helpfulness_votes_lst.extend(Helpfulness_votes_lst)

        next_page_buton = driver.find_element(By.XPATH,"//a[@data-track='Page next']")
        next_page_buton.click()

    driver.quit()


ElementClickInterceptedException: Message: element click intercepted: Element <a aria-disabled="false" class="" data-track="Page next" href="/site/reviews/samsung-galaxy-s23-ultra-256gb-unlocked-phantom-black/6529723?variant=A&amp;pageSize=20&amp;sku=6529723&amp;sort=BEST_REVIEW&amp;variant=A&amp;page=2" role="button" title="next Page">...</a> is not clickable at point (1668, 896). Other element would receive the click: <div id="survey_window" class="modal fade modal-open in" role="dialog" style="margin-top: 331px; display: block;">...</div>
  (Session info: chrome=122.0.6261.112)
Stacktrace:
	GetHandleVerifier [0x00007FF657D75E42+3538674]
	(No symbol) [0x00007FF657994C02]
	(No symbol) [0x00007FF657845AEB]
	(No symbol) [0x00007FF6578933E4]
	(No symbol) [0x00007FF657891122]
	(No symbol) [0x00007FF65788E884]
	(No symbol) [0x00007FF65788D5A9]
	(No symbol) [0x00007FF657881950]
	(No symbol) [0x00007FF6578AF0AA]
	(No symbol) [0x00007FF65788124A]
	(No symbol) [0x00007FF6578AF2C0]
	(No symbol)

In [6]:
product_data = {
    'Name': Global_product_name_lst,  
    'Star_rating': Global_product_star_rating_lst,
    'Total_reviews': Global_product_total_reviews
}

df_product = pd.DataFrame(product_data)
df_product

,Name,Star_rating,Total_reviews
0,Apple - iPhone 14 128GB (Unlocked) - Midnight,4.9,209
1,Apple - iPhone 14 128GB (Unlocked) - Midnight,4.9,209
2,Samsung - Galaxy S22 Ultra 128GB - Phantom Bla...,4.6,396
3,HP - DeskJet 2755e Wireless Inkjet Printer wit...,4.1,3489
4,JBL - Flip 5 Portable Bluetooth Speaker - Black,4.8,5932
5,Microsoft - 365 Family (Up to 6 People) (12-Mo...,4.5,10353
6,TurboTax - Deluxe 2023 Federal + E-file & Stat...,4.7,286
7,Hamilton Beach - Classic 4 Slice Toaster with ...,4.5,28
8,Razor - MX350 Dirt Rocket eBike w/7 miles Max ...,4.4,199
9,"LG - 65"" Class C3 Series OLED 4K UHD Smart web...",4.8,1470


In [14]:
df_product_without_duplicates = df_product.drop_duplicates()

df_products = df_product_without_duplicates.reset_index(drop=True)
df_products

,Name,Star_rating,Total_reviews
0,Apple - iPhone 14 128GB (Unlocked) - Midnight,4.9,209
1,Samsung - Galaxy S22 Ultra 128GB - Phantom Bla...,4.6,396
2,HP - DeskJet 2755e Wireless Inkjet Printer wit...,4.1,3489
3,JBL - Flip 5 Portable Bluetooth Speaker - Black,4.8,5932
4,Microsoft - 365 Family (Up to 6 People) (12-Mo...,4.5,10353
5,TurboTax - Deluxe 2023 Federal + E-file & Stat...,4.7,286
6,Hamilton Beach - Classic 4 Slice Toaster with ...,4.5,28
7,Razor - MX350 Dirt Rocket eBike w/7 miles Max ...,4.4,199
8,"LG - 65"" Class C3 Series OLED 4K UHD Smart web...",4.8,1470
9,GE - 1.4 Cu. Ft. Mid-Size Microwave - Black,4.0,860


In [7]:
review_data = {
    'Reviewer_Name':Global_Reviewer_name_lst,
    'Review_Title':Global_Review_title_lst,
    'Star_Rating':Global_Reviewer_star_rating_lst,
    'Review_Content':Global_Review_content_lst,
    'Helpfulness_votes':Global_Helpfulness_votes_lst
}

df_review = pd.DataFrame(review_data)
df_review

,Reviewer_Name,Review_Title,Star_Rating,Review_Content,Helpfulness_votes
0,AmandaD,Awesome experience all around,5,"Husband upgraded from a 12 to a 14, loves it. ...",0
1,AmandaD,Awesome experience all around,5,"Husband upgraded from a 12 to a 14, loves it. ...",0
2,MauraC,Great phone,5,It’s a pretty great phone. It has all that I n...,0
3,AmandaD,Awesome experience all around,5,"Husband upgraded from a 12 to a 14, loves it. ...",0
4,MauraC,Great phone,5,It’s a pretty great phone. It has all that I n...,0
...,...,...,...,...,...
79441236,Review,"Great features, amazing camera!",5,I can confidently say that it is an incredible...,6
79441237,SaravananD,After 3 weeks of useing S23 Ultra Phone!!!,5,The best phone EVER!!\nThe Camera:\nAmazing it...,0
79441238,Anonymous,Superb flagship phone!,5,This is another superb flagship from Samsung! ...,0
79441239,ErnieG,Excellent phone,5,I have tried to hold on to my old phone for a ...,0


In [8]:
len(df_review)

79441241

In [11]:
df_review_without_duplicates = df_review.drop_duplicates()
df_review_without_duplicates.head()

,Reviewer_Name,Review_Title,Star_Rating,Review_Content,Helpfulness_votes
0,AmandaD,Awesome experience all around,5,"Husband upgraded from a 12 to a 14, loves it. ...",0
2,MauraC,Great phone,5,It’s a pretty great phone. It has all that I n...,0
5,Peachy,It’s an iPhone,5,"It’s a unlocked iPhone, I was able to pick up ...",0
9,PauletteP,Hello….Can You Hear Me Now?,5,Apple products are very good products but of c...,0
14,AnthonyC,Great!!,5,Awesome product love it so much. I would recom...,0


In [10]:
df_review_without_duplicates

,Reviewer_Name,Review_Title,Star_Rating,Review_Content,Helpfulness_votes
0,AmandaD,Awesome experience all around,5,"Husband upgraded from a 12 to a 14, loves it. ...",0
2,MauraC,Great phone,5,It’s a pretty great phone. It has all that I n...,0
5,Peachy,It’s an iPhone,5,"It’s a unlocked iPhone, I was able to pick up ...",0
9,PauletteP,Hello….Can You Hear Me Now?,5,Apple products are very good products but of c...,0
14,AnthonyC,Great!!,5,Awesome product love it so much. I would recom...,0
...,...,...,...,...,...
20120,BrendajW,Excellent phone,5,Excellent phone and the camera is great!!! Ver...,1
20127,Krusaderzcom,Samsung Galaxy S22 Ultra,5,I had one for personal use and had to get a ph...,0
20135,Spiderman,Phone,5,"one of my favorite phones. Love the size, Came...",1
20144,Grifter,Love my new phone,5,I love the camera on this phone. SIt is real n...,0


In [12]:
df_reviews = df_review_without_duplicates.reset_index(drop=True)
df_reviews.head()

,Reviewer_Name,Review_Title,Star_Rating,Review_Content,Helpfulness_votes
0,AmandaD,Awesome experience all around,5,"Husband upgraded from a 12 to a 14, loves it. ...",0
1,MauraC,Great phone,5,It’s a pretty great phone. It has all that I n...,0
2,Peachy,It’s an iPhone,5,"It’s a unlocked iPhone, I was able to pick up ...",0
3,PauletteP,Hello….Can You Hear Me Now?,5,Apple products are very good products but of c...,0
4,AnthonyC,Great!!,5,Awesome product love it so much. I would recom...,0


In [15]:
df_products.to_excel("BB_Product_data_all.xlsx", index=False)
df_reviews.to_excel("BB_Review_data_all.xlsx", index=False)